In [61]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types 
from pyspark.sql.window import Window

In [62]:
# spark = SparkSession.builder.appName("covid").getOrCreate()
spark = SparkSession.builder \
    .appName("covid") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()
filepath = "/Users/VC/Desktop/covid_task/complete.csv"






In [63]:
df_csv = spark.read.format("csv") \
            .option("header", True) \
            .option("multiLine", True) \
            .option("ignoreLeadingWhiteSpace",True) \
            .option("ignoreTrailingWhiteSpace",True) \
            .option("escape", "\\") \
            .option("quote", "\"") \
            .load(filepath)

In [64]:
df_csv.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Name of State / UT: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Total Confirmed cases: string (nullable = true)
 |-- Death: string (nullable = true)
 |-- Cured/Discharged/Migrated: string (nullable = true)
 |-- New cases: string (nullable = true)
 |-- New deaths: string (nullable = true)
 |-- New recovered: string (nullable = true)



In [65]:
df_csv = df_csv.withColumn("total_case", df_csv["Total Confirmed cases"].cast(types.LongType()))
df_csv = df_csv.withColumn("total_newly_recovered", df_csv["New recovered"].cast(types.LongType()))
df_csv = df_csv.withColumn("state", df_csv["Name of State / UT"].cast(types.StringType()))
df_csv = df_csv.withColumn("death_Case", df_csv["Death"].cast(types.LongType()))
df_csv.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Name of State / UT: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Total Confirmed cases: string (nullable = true)
 |-- Death: string (nullable = true)
 |-- Cured/Discharged/Migrated: string (nullable = true)
 |-- New cases: string (nullable = true)
 |-- New deaths: string (nullable = true)
 |-- New recovered: string (nullable = true)
 |-- total_case: long (nullable = true)
 |-- total_newly_recovered: long (nullable = true)
 |-- state: string (nullable = true)
 |-- death_Case: long (nullable = true)



# The day had more number of covid cases.

In [66]:
output_df_1 = df_csv.groupBy("Date").agg(F.sum("total_case").alias("sum_total_case"))

In [67]:
window_spec = Window.orderBy(F.col("sum_total_case").desc())

In [68]:
output_df_1 = output_df_1.withColumn("recency", F.row_number().over(window_spec)).filter(F.col('recency') == 1).drop('recency')

In [69]:
output_df_1.show()

+----------+--------------+
|      Date|sum_total_case|
+----------+--------------+
|2020-08-06|       1964536|
+----------+--------------+



# The state has the second-largest number of covid cases.

In [70]:
output_df_2 = df_csv.groupBy("state").agg(F.sum("total_case").alias("sum_total_case"))

In [71]:
window_spec = Window.orderBy(F.col("sum_total_case").desc())

In [72]:
output_df_2 = output_df_2.withColumn("recency", F.row_number().over(window_spec)).filter(F.col('recency') == 2).drop('recency')

In [73]:
output_df_2.show()

+----------+--------------+
|     state|sum_total_case|
+----------+--------------+
|Tamil Nadu|       7847083|
+----------+--------------+



# Which Union Territory has the least number of death.

In [74]:
output_df_3 = df_csv.where(F.col('state').like("Union Territory%"))

In [75]:
output_df_3 = output_df_3.groupBy("state").agg(F.sum("death_Case").alias("sum_total_death"))

In [76]:
window_spec = Window.orderBy(F.col("sum_total_death"))

In [77]:
output_df_3 = output_df_3.withColumn("recency", F.row_number().over(window_spec)).filter(F.col('recency') == 1).drop('recency')

In [78]:
output_df_3.show(truncate=False)

+------------------------------------+---------------+
|state                               |sum_total_death|
+------------------------------------+---------------+
|Union Territory of Jammu and Kashmir|0              |
+------------------------------------+---------------+



# The state has the Lowest Death to Total Confirmed cases ratio.

In [79]:
output_df_4 = df_csv.withColumn("ratio", F.col("death_Case")/F.col("total_case"))

In [80]:
output_df_4 = output_df_4.groupBy("state").agg(F.avg("ratio").alias("avg_ratio"))

In [81]:
window_spec = Window.orderBy(F.col("avg_ratio"))

In [82]:
output_df_4 = output_df_4.withColumn("recency", F.row_number().over(window_spec)).filter(F.col('recency') == 1).drop('recency')

In [83]:
output_df_4.show(truncate=False)

+-------+---------+
|state  |avg_ratio|
+-------+---------+
|Mizoram|0.0      |
+-------+---------+



# Convert all state names to lowercase.

In [84]:
output_df_5 = df_csv.withColumn('state_lower', F.lower(F.col("state")))

In [85]:
output_df_5.select("state_lower").distinct().show()

+--------------------+
|         state_lower|
+--------------------+
|               delhi|
|         maharashtra|
|           meghalaya|
|              odisha|
|             haryana|
|         west bengal|
|                 goa|
|              punjab|
|   jammu and kashmir|
|dadra and nagar h...|
|           karnataka|
|      andhra pradesh|
|           telangana|
|            nagaland|
|               bihar|
|      madhya pradesh|
|           jharkhand|
|               assam|
|              kerala|
|          tamil nadu|
+--------------------+
only showing top 20 rows



# Find which month the Newer recovered cases.

In [86]:
import calendar
from datetime import datetime

def get_month_name(month_number):
    return calendar.month_name[int(month_number)]

def get_month(date):
    print(date)
    date = datetime.strptime(date, "%Y-%m-%d")
    return date.month

get_month_udf = F.udf(lambda a : get_month(a), types.StringType())
get_month_name_udf = F.udf(lambda a : get_month_name(a), types.StringType())

In [87]:
output_df_6 = df_csv.withColumn("month", get_month_udf(F.col("date")))

In [88]:
output_df_6 = output_df_6.groupBy("month").agg(F.sum("total_newly_recovered").alias("sum_newly_recovered"))

In [89]:
window_spec = Window.orderBy(F.col("sum_newly_recovered").desc())

In [90]:
output_df_6 = output_df_6.withColumn("recency", F.row_number().over(window_spec)).filter(F.col('recency') == 1).drop('recency')

In [91]:
output_df_6 = output_df_6.withColumn("month", get_month_name_udf(F.col("month")))

In [92]:
output_df_6.printSchema()


root
 |-- month: string (nullable = true)
 |-- sum_newly_recovered: long (nullable = true)



In [96]:
output_df_6.show(5)
output_df_6.show(truncate=3)
output_df_6.show(vertical=True)


Py4JJavaError: An error occurred while calling o611.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 48.0 failed 1 times, most recent failure: Lost task 0.0 in stage 48.0 (TID 36) (192.168.1.36 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(Int